In [4]:
import pandas as pd
import xlsxwriter

In [8]:
file_path = 'Business Data Sample.xlsx'
data = pd.read_excel(file_path, sheet_name='Main')

In [10]:
# Clean the data: Ensure quantity is positive and calculate revenue
data['quantity'] = data['quantity'].abs()
data['revenue'] = data['quantity'] * data['amount']

In [11]:
# File Path for Output
output_file = 'Processed_Business_Data.xlsx'

In [12]:
# Create an Excel file with analyses
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    workbook = writer.book

    # Save the original data with calculations
    data.to_excel(writer, sheet_name='Cleaned Data', index=False)
    worksheet_cleaned = writer.sheets['Cleaned Data']

    # Apply Conditional Formatting: Highlight high revenue transactions
    high_revenue_format = workbook.add_format({'bg_color': '#FFEB9C', 'font_color': '#9C6500'})
    worksheet_cleaned.conditional_format('L2:L1048576', {
        'type': 'cell',
        'criteria': '>',
        'value': 1000,
        'format': high_revenue_format
    })

    # 1. Total Revenue by Category and Brand
    revenue_by_category_brand = data.groupby(['Category', 'brand'])['revenue'].sum().reset_index()
    revenue_by_category_brand.to_excel(writer, sheet_name='Category_Brand_Revenue', index=False)

    # 2. Top 5 Customers by Revenue
    top_customers = data.groupby(['customer_name', 'customer_code'])['revenue'].sum().nlargest(5).reset_index()
    top_customers.to_excel(writer, sheet_name='Top_Customers', index=False)

    # 3. Most Frequently Sold Product by Customer Type
    most_sold_product_per_customer_type = (
        data.groupby(['customer_type', 'upc (Unit per Case)'])['quantity'].sum()
        .reset_index()
        .sort_values(['customer_type', 'quantity'], ascending=[True, False])
        .drop_duplicates(subset=['customer_type'], keep='first')
    )
    most_sold_product_per_customer_type.to_excel(writer, sheet_name='Most_Sold_Products', index=False)

    # 4. City-Wise Trends: Revenue and Transaction Count
    city_revenue = data.groupby('City')['revenue'].sum().reset_index()
    city_transactions = data['City'].value_counts().reset_index()
    city_transactions.columns = ['City', 'Transaction Count']
    city_revenue.to_excel(writer, sheet_name='City_Revenue', index=False)
    city_transactions.to_excel(writer, sheet_name='City_Transactions', index=False)

    # 5. Average Quantity per Transaction for Each Channel
    avg_quantity_per_channel = data.groupby('channel_description')['quantity'].mean().reset_index()
    avg_quantity_per_channel.to_excel(writer, sheet_name='Avg_Quantity_Channel', index=False)

    # 6. Top Reasons for Transactions and Their Revenue Impact
    reason_impact = data.groupby('reason for transaction')['revenue'].sum().reset_index()
    reason_impact.to_excel(writer, sheet_name='Reason_Impact', index=False)

    # 7. Pivot Table: City-wise Revenue and Quantity Trends
    pivot_city_revenue_quantity = data.pivot_table(
        index='City', values=['revenue', 'quantity'], aggfunc='sum'
    ).reset_index()
    pivot_city_revenue_quantity.to_excel(writer, sheet_name='Pivot_City_Trends', index=False)

# Inform about completion
print(f"Processed Excel file has been saved as {output_file}.")

Processed Excel file has been saved as Processed_Business_Data.xlsx.
